#Pandas, SQL, and the Grammar of Data

---

##Table of Contents
* [Pandas, SQL, and the Grammar of Data](#Pandas,-SQL,-and-the-Grammar-of-Data)
	* [Populating the Database](#Populating-the-Database)
		* [SQLITE](#SQLITE)
		* [Init](#Init)
			* [Populating with Pandas!!](#Populating-with-Pandas!!)
			* [Or populate with SQL INSERT](#Or-populate-with-SQL-INSERT)
			* [Bulk insert](#Bulk-insert)
	* [Single Table Verbs](#Single-Table-Verbs)
		* [QUERY](#QUERY)
		* [SORT](#SORT)
		* [SELECT-COLUMNS](#SELECT-COLUMNS)
		* [SELECT-DISTINCT](#SELECT-DISTINCT)
		* [ASSIGN](#ASSIGN)
		* [AGGREGATE](#AGGREGATE)
		* [GROUP-AGG](#GROUP-AGG)
		* [DELETE](#DELETE)
		* [LIMIT](#LIMIT)
	* [Indexes](#Indexes)
	* [Relationships: JOINs are Cartesian Products.](#Relationships:-JOINs-are-Cartesian-Products.)
		* [Simple subselect](#Simple-subselect)
		* [implicit join](#implicit-join)
		* [Explicit INNER JOIN](#Explicit-INNER-JOIN)
		* [Outer JOIN](#Outer-JOIN)
			* [left outer (contributors on candidates)](#left-outer-%28contributors-on-candidates%29)
			* [right outer (contributors on candidates) = left outer (candidates on contributors)](#right-outer-%28contributors-on-candidates%29-=-left-outer-%28candidates-on-contributors%29)
			* [full outer](#full-outer)
	* [Pandas /SQL](#Pandas-/SQL)
	* [Useful Links](#Useful-Links)


In [1]:
# The %... is an iPython thing, and is not part of the Python language.
# In this case we're just telling the plotting library to draw things on
# the notebook, instead of on a separate window.
%matplotlib inline
# See all the "as ..." contructs? They're just aliasing the package names.
# That way we can call methods like plt.plot() instead of matplotlib.pyplot.plot().
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

##Populating the Database

Lets start with Relational Databases, so called because they contain "relations" (tables), which are SETS of "tuples" (rows) which map "attributes" to atomic values.

The available attributes are constrained by a "header" tuple of attributes which set the type. We do this below here.

In [2]:
ourschema="""
DROP TABLE IF EXISTS "candidates";
DROP TABLE IF EXISTS "contributors";
CREATE TABLE "candidates" (
    "id" INTEGER PRIMARY KEY  NOT NULL ,
    "first_name" VARCHAR,
    "last_name" VARCHAR,
    "middle_name" VARCHAR,
    "party" VARCHAR NOT NULL
);
CREATE TABLE "contributors" (
    "id" INTEGER PRIMARY KEY  AUTOINCREMENT  NOT NULL,
    "last_name" VARCHAR,
    "first_name" VARCHAR,
    "middle_name" VARCHAR,
    "street_1" VARCHAR,
    "street_2" VARCHAR,
    "city" VARCHAR,
    "state" VARCHAR,
    "zip" VARCHAR,
    "amount" INTEGER,
    "date" DATETIME,
    "candidate_id" INTEGER NOT NULL,
    FOREIGN KEY(candidate_id) REFERENCES candidates(id)
);
"""

### SQLITE

We use sqlite here (and recommend Postgres for production purposes). Still sqlite is great for on-disk large databases which wont fit into memory. 

Its also built into Python, but to use the [command line tool](https://www.sqlite.org/cli.html), I recommend you install it: https://www.sqlite.org/download.html. I also recommend you download and install the sqlite browser: http://sqlitebrowser.org .

Python implements a standard database API over all databases. Its called [DBAPI2](http://cewing.github.io/training.codefellows/lectures/day21/intro_to_dbapi2.html). It works across many SQL databases.

There is an even higher level API available, called [SQLAlchemy](http://www.sqlalchemy.org). While we wont use it here, I thoroughly recommend it, either in its direct relational form, or ORM form. Many things in Pandas use it to interface with databases. Here we'll get away with things by using SQLITE.

---

Sqlite is a text or memory based database. Connect and get a DBAPI2 connection.

In [3]:
from sqlite3 import dbapi2 as sq3
import os
PATHSTART="."
def get_db(dbfile):
    sqlite_db = sq3.connect(os.path.join(PATHSTART, dbfile))
    return sqlite_db

Drop tables if they exist and create them.

In [4]:
def init_db(dbfile, schema):
    """Creates the database tables."""
    db = get_db(dbfile)
    db.cursor().executescript(schema)
    db.commit()
    return db

Use Pandas to read in the data

In [5]:
dfcand=pd.read_csv("./candidates.txt", sep='|')
dfcand

,id,first_name,last_name,middle_name,party
0,33,Joseph,Biden,NaN,D
1,36,Samuel,Brownback,NaN,R
2,34,Hillary,Clinton,R.,D
3,39,Christopher,Dodd,J.,D
4,26,John,Edwards,NaN,D
5,22,Rudolph,Giuliani,NaN,R
6,24,Mike,Gravel,NaN,D
7,16,Mike,Huckabee,NaN,R
8,30,Duncan,Hunter,NaN,R
9,31,Dennis,Kucinich,NaN,D


In [9]:
dfcwci=pd.read_csv("./contributors_with_candidate_id.txt", sep="|")
dfcwci.head()

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,NaN,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,NaN,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


In [7]:
del dfcwci['id']
dfcwci.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


###Init

Initializing the database

In [6]:
db=init_db("cancont.db", ourschema)

####Populating with Pandas!!

In [7]:
dfcand.to_sql("candidates", db, if_exists="append", index=False)

In [10]:
dfcwci.to_sql("contributors", db, if_exists="append", index=False)

In [11]:
dfcwci.shape

(175, 12)

In [12]:
sel="""
SELECT * FROM candidates;
"""
c=db.cursor().execute(sel)

In [13]:
c.fetchall()

[(16, 'Mike', 'Huckabee', None, 'R'),
 (20, 'Barack', 'Obama', None, 'D'),
 (22, 'Rudolph', 'Giuliani', None, 'R'),
 (24, 'Mike', 'Gravel', None, 'D'),
 (26, 'John', 'Edwards', None, 'D'),
 (29, 'Bill', 'Richardson', None, 'D'),
 (30, 'Duncan', 'Hunter', None, 'R'),
 (31, 'Dennis', 'Kucinich', None, 'D'),
 (32, 'Ron', 'Paul', None, 'R'),
 (33, 'Joseph', 'Biden', None, 'D'),
 (34, 'Hillary', 'Clinton', 'R.', 'D'),
 (35, 'Mitt', 'Romney', None, 'R'),
 (36, 'Samuel', 'Brownback', None, 'R'),
 (37, 'John', 'McCain', None, 'R'),
 (38, 'Tom', 'Tancredo', None, 'R'),
 (39, 'Christopher', 'Dodd', 'J.', 'D'),
 (41, 'Fred', 'Thompson', 'D.', 'R')]

In [14]:
rem="""
DELETE FROM candidates;
"""
c=db.cursor().execute(rem)
db.commit()

In [15]:
c.fetchall()

[]

####Or populate with SQL INSERT

![xkcd-sqlinj](http://imgs.xkcd.com/comics/exploits_of_a_mom.png)

In [17]:
ins="""
INSERT INTO candidates (id, first_name, last_name, middle_name, party) \
    VALUES (?,?,?,?,?);
"""
with open("candidates.txt") as fd:
    slines =[l.strip().split('|') for l in fd.readlines()]
    for line in slines[1:]:
        theid, first_name, last_name, middle_name, party = line
        print (theid, first_name, last_name, middle_name, party)
        valstoinsert = (int(theid), first_name, last_name, middle_name, party)
        print (ins, valstoinsert)
        db.cursor().execute(ins, valstoinsert)
        
    

33 Joseph Biden  D

INSERT INTO candidates (id, first_name, last_name, middle_name, party)     VALUES (?,?,?,?,?);
 (33, 'Joseph', 'Biden', '', 'D')
36 Samuel Brownback  R

INSERT INTO candidates (id, first_name, last_name, middle_name, party)     VALUES (?,?,?,?,?);
 (36, 'Samuel', 'Brownback', '', 'R')
34 Hillary Clinton R. D

INSERT INTO candidates (id, first_name, last_name, middle_name, party)     VALUES (?,?,?,?,?);
 (34, 'Hillary', 'Clinton', 'R.', 'D')
39 Christopher Dodd J. D

INSERT INTO candidates (id, first_name, last_name, middle_name, party)     VALUES (?,?,?,?,?);
 (39, 'Christopher', 'Dodd', 'J.', 'D')
26 John Edwards  D

INSERT INTO candidates (id, first_name, last_name, middle_name, party)     VALUES (?,?,?,?,?);
 (26, 'John', 'Edwards', '', 'D')
22 Rudolph Giuliani  R

INSERT INTO candidates (id, first_name, last_name, middle_name, party)     VALUES (?,?,?,?,?);
 (22, 'Rudolph', 'Giuliani', '', 'R')
24 Mike Gravel  D

INSERT INTO candidates (id, first_name, last_name

In [18]:
def make_query(sel):
    c=db.cursor().execute(sel)
    return c.fetchall()

In [19]:
make_query("SELECT * FROM candidates;")

[(16, 'Mike', 'Huckabee', '', 'R'),
 (20, 'Barack', 'Obama', '', 'D'),
 (22, 'Rudolph', 'Giuliani', '', 'R'),
 (24, 'Mike', 'Gravel', '', 'D'),
 (26, 'John', 'Edwards', '', 'D'),
 (29, 'Bill', 'Richardson', '', 'D'),
 (30, 'Duncan', 'Hunter', '', 'R'),
 (31, 'Dennis', 'Kucinich', '', 'D'),
 (32, 'Ron', 'Paul', '', 'R'),
 (33, 'Joseph', 'Biden', '', 'D'),
 (34, 'Hillary', 'Clinton', 'R.', 'D'),
 (35, 'Mitt', 'Romney', '', 'R'),
 (36, 'Samuel', 'Brownback', '', 'R'),
 (37, 'John', 'McCain', '', 'R'),
 (38, 'Tom', 'Tancredo', '', 'R'),
 (39, 'Christopher', 'Dodd', 'J.', 'D'),
 (41, 'Fred', 'Thompson', 'D.', 'R')]

In [20]:
rem="""
DELETE FROM candidates;
"""
c=db.cursor().execute(rem)
db.commit()

####Bulk insert

You almost always have to do this from the command line. Its typically faster, but also different foe every database

In [ ]:
%%bash
tail -n +2 candidates.txt > candidates_nohead.txt
echo ".import candidates_nohead.txt candidates" | sqlite3 cancont.db

In [ ]:
make_query("SELECT * FROM candidates;")

##Single Table Verbs

Let us now focus on core data manipulation commands. The reason to do this is that they are *universal across systems, and by identifying them, we can quickly ask how to do these* when we encounter a new system.

See https://gist.github.com/TomAugspurger/6e052140eaa5fdb6e8c0/ which has a comparison of r/dplyr and pandas. I stole and modified this table from there:

``dplyr`` has a small set of nicely defined verbs. I've listed their closest pandas verbs.


<table>
  <tr>
    <th><b>VERB</b></th>
    <th><b>dplyr</b></th>
    <th><b>pandas</b></th>
    <th><b>SQL</b></th>
  </tr>
  <tr>
    <td>QUERY/SELECTION</td>
    <td>filter() (and slice())</td>
    <td>query() (and loc[], iloc[])</td>
    <td>SELECT WHERE</td>
  </tr>
  <tr>
    <td>SORT</td>
    <td>arrange()</td>
    <td>sort()</td>
    <td>ORDER BY</td>
  </tr>
  <tr>
    <td>SELECT-COLUMNS/PROJECTION</td>
    <td>select() (and rename())</td>
    <td>[](__getitem__) (and rename())</td>
    <td>SELECT COLUMN</td>
  </tr>
  <tr>
    <td>SELECT-DISTINCT</td>
    <td>distinct()</td>
    <td>unique(),drop_duplicates()</td>
    <td>SELECT DISTINCT COLUMN</td>
  </tr>
  <tr>
    <td>ASSIGN</td>
    <td>mutate() (and transmute())</td>
    <td>assign</td>
    <td>ALTER/UPDATE</td>
  </tr>
  <tr>
    <td>AGGREGATE</td>
    <td>summarise()</td>
    <td>describe(), mean(), max()</td>
    <td>None, AVG(),MAX()</td>
  </tr>
  <tr>
    <td>SAMPLE</td>
    <td>sample_n() and sample_frac()</td>
    <td>sample()</td>
    <td>implementation dep, use RAND()</td>
  </tr>
  <tr>
    <td>GROUP-AGG</td>
    <td>group_by/summarize</td>
    <td>groupby/agg, count, mean</td>
    <td>GROUP BY</td>
  </tr>
  <tr>
    <td>DELETE</td>
    <td>?</td>
    <td>drop/masking</td>
    <td>DELETE/WHERE</td>
  </tr>
</table>


In [14]:
dfcwci.head()

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,NaN,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,NaN,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


###QUERY

In [15]:
dfcwci.query("state=='VA' & amount < 400")

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
27,NaN,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.00,2007-09-19,20
77,NaN,Ranganath,Anoop,NaN,2507 Willard Drive,NaN,Charlottesville,VA,22903,-100.00,2008-04-21,32
88,NaN,Perreault,Louise,NaN,503 Brockridge Hunt Drive,NaN,Hampton,VA,23666,-34.08,2008-04-21,32
145,NaN,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,NaN,FAIRFAX,VA,220308440,50.00,2007-09-30,35


In [16]:
dfcwci[(dfcwci.state=='VA') & (dfcwci.amount < 400)]

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
27,NaN,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.00,2007-09-19,20
77,NaN,Ranganath,Anoop,NaN,2507 Willard Drive,NaN,Charlottesville,VA,22903,-100.00,2008-04-21,32
88,NaN,Perreault,Louise,NaN,503 Brockridge Hunt Drive,NaN,Hampton,VA,23666,-34.08,2008-04-21,32
145,NaN,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,NaN,FAIRFAX,VA,220308440,50.00,2007-09-30,35


In [17]:
cont_cols = [e[1] for e in make_query("PRAGMA table_info(contributors);")]
cont_cols

NameError: name 'make_query' is not defined

In [30]:
def make_frame(list_of_tuples, legend=cont_cols):
    framelist=[]
    for i, cname in enumerate(legend):
        framelist.append((cname,[e[i] for e in list_of_tuples]))
    return pd.DataFrame.from_items(framelist)

In [32]:
out=make_query("SELECT * FROM contributors WHERE state='VA' AND amount < 400;")
print(out)
make_frame(out)

[(28, 'Buckheit', 'Bruce', None, '8904 KAREN DR', None, 'FAIRFAX', 'VA', '220312731', 100, '2007-09-19', 20), (78, 'Ranganath', 'Anoop', None, '2507 Willard Drive', None, 'Charlottesville', 'VA', '22903', -100, '2008-04-21', 32), (89, 'Perreault', 'Louise', None, '503 Brockridge Hunt Drive', None, 'Hampton', 'VA', '23666', -34.08, '2008-04-21', 32), (146, 'ABDELLA', 'THOMAS', 'M.', '4231 MONUMENT WALL WAY #340', None, 'FAIRFAX', 'VA', '220308440', 50, '2007-09-30', 35), (203, 'Buckheit', 'Bruce', None, '8904 KAREN DR', None, 'FAIRFAX', 'VA', '220312731', 100, '2007-09-19', 20), (253, 'Ranganath', 'Anoop', None, '2507 Willard Drive', None, 'Charlottesville', 'VA', '22903', -100, '2008-04-21', 32), (264, 'Perreault', 'Louise', None, '503 Brockridge Hunt Drive', None, 'Hampton', 'VA', '23666', -34.08, '2008-04-21', 32), (321, 'ABDELLA', 'THOMAS', 'M.', '4231 MONUMENT WALL WAY #340', None, 'FAIRFAX', 'VA', '220308440', 50, '2007-09-30', 35)]


,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,28,Buckheit,Bruce,None,8904 KAREN DR,None,FAIRFAX,VA,220312731,100.00,2007-09-19,20
1,78,Ranganath,Anoop,None,2507 Willard Drive,None,Charlottesville,VA,22903,-100.00,2008-04-21,32
2,89,Perreault,Louise,None,503 Brockridge Hunt Drive,None,Hampton,VA,23666,-34.08,2008-04-21,32
3,146,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,None,FAIRFAX,VA,220308440,50.00,2007-09-30,35
4,203,Buckheit,Bruce,None,8904 KAREN DR,None,FAIRFAX,VA,220312731,100.00,2007-09-19,20
5,253,Ranganath,Anoop,None,2507 Willard Drive,None,Charlottesville,VA,22903,-100.00,2008-04-21,32
6,264,Perreault,Louise,None,503 Brockridge Hunt Drive,None,Hampton,VA,23666,-34.08,2008-04-21,32
7,321,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,None,FAIRFAX,VA,220308440,50.00,2007-09-30,35


In [33]:
out=make_query("SELECT * FROM contributors WHERE state IS NULL;")
make_frame(out)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,126,BOURNE,TRAVIS,None,LAGE KAART 77,None,BRASSCHATT,None,2930,-500,2008-11-20,35
1,301,BOURNE,TRAVIS,None,LAGE KAART 77,None,BRASSCHATT,None,2930,-500,2008-11-20,35


In [34]:
dfcwci[dfcwci.state.isnull()]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
125,BOURNE,TRAVIS,NaN,LAGE KAART 77,NaN,BRASSCHATT,NaN,2930,-500.0,2008-11-20,35


In [35]:
out=make_query("SELECT * FROM contributors WHERE state IS NOT NULL;")
make_frame(out).shape

(348, 12)

In [36]:
dfcwci[dfcwci.state.notnull()].shape

(174, 11)

In [ ]:
out=make_query("SELECT * FROM contributors WHERE state IN ('VA','WA');")
make_frame(out).head(10)

In [ ]:
dfcwci[dfcwci.state.isin(['VA','WA'])].head(10)

In [ ]:
out=make_query("SELECT * FROM contributors WHERE amount BETWEEN 10 AND 50;")
make_frame(out).head(10)

In [ ]:
dfcwci.query("10 <= amount <= 50").head(10)

###SORT

In [ ]:
dfcwci.sort("amount").head(10)

In [ ]:
dfcwci.sort("amount", ascending=False).head(10)

In [ ]:
out=make_query("SELECT * FROM contributors ORDER BY amount;")
make_frame(out).head(10)

In [ ]:
out=make_query("SELECT * FROM contributors ORDER BY amount DESC;")
make_frame(out).head(10)

###SELECT-COLUMNS

In [ ]:
dfcwci[['first_name', 'amount']].head(10)

In [ ]:
out=make_query("SELECT first_name, amount FROM contributors;")
make_frame(out,['first_name', 'amount']).head(10)

###SELECT-DISTINCT

In [ ]:
dfcwci[['last_name','first_name']].count()

In [ ]:
dfcwci[['last_name','first_name']].drop_duplicates().count()

In [ ]:
dfcwci[['last_name','first_name']].drop_duplicates().head(10)

In [ ]:
out=make_query("SELECT DISTINCT last_name, first_name FROM contributors;")
make_frame(out,['last_name', 'first_name']).head(10)

###ASSIGN

In [ ]:
dfcwci['name']=dfcwci['last_name']+", "+dfcwci['first_name']
dfcwci.head(10)

In [ ]:
dfcwci.assign(ucname=dfcwci.last_name+":"+dfcwci.first_name).head(10)

Will the above command actually change `dfcwci`?

####What if we wanted to change an existing assignment?

In [ ]:
dfcwci[dfcwci.state=='VA']

In [ ]:
dfcwci.loc[dfcwci.state=='VA', 'name']

In [ ]:
dfcwci.loc[dfcwci.state=='VA', 'name']="junk"

In [ ]:
dfcwci.query("state=='VA'")

---

Let us see the entire process in SQL

In [ ]:
alt="ALTER TABLE contributors ADD COLUMN name;"
db.cursor().execute(alt)

In [ ]:
make_query("PRAGMA table_info(contributors);")

In [ ]:
out = make_query("SELECT id, last_name,first_name from contributors;")
out2 = [(e[1]+", "+e[2],e[0]) for e in out]
out2

In [ ]:
alt2="UPDATE contributors SET name = ? WHERE id = ?;"
for ele in out2:
    db.cursor().execute(alt2, ele)

In [ ]:
db.commit()

In [ ]:
out=make_query("SELECT * from contributors;")
make_frame(out,cont_cols+["name"]).head(10)

And lets now do an assignment to an existing column

In [ ]:
upd="UPDATE contributors SET name = 'junk' WHERE state = 'VA';"
db.cursor().execute(upd)
db.commit()

In [ ]:
out=make_query("SELECT * from contributors where state='VA';")
make_frame(out,cont_cols+["name"]).head(10)

####No DROP COLUMN in SQLITE

Its available in other databases. Here you must just re-create your database, or no about this gotcha from the start.

In [ ]:
alt="ALTER TABLE contributors DROP COLUMN name;"
db.cursor().execute(alt)
db.commit()

Its much simpler in Pandas, of-course

In [ ]:
del dfcwci['name']

###AGGREGATE

In [ ]:
dfcwci.describe()

In [ ]:
dfcwci.amount.max()

In [ ]:
dfcwci[dfcwci.amount==dfcwci.amount.max()]

In [ ]:
out=make_query("SELECT *, MAX(amount) AS maxamt FROM contributors;")
print out
make_frame(out, cont_cols+['maxamt'])

In [ ]:
out=make_query("SELECT COUNT(amount) AS AMOUNTCOUNT FROM contributors;")
print out

In [ ]:
out=make_query("SELECT AVG(amount) FROM contributors;")
print out

In [ ]:
dfcwci[dfcwci.amount > dfcwci.amount.max() - 2300]

In [ ]:
out=make_query("SELECT * FROM contributors WHERE amount > (select (MAX(amount) - 2300) FROM contributors);")
make_frame(out)

Aso `MIN`, `SUM`, `AVG`.

###GROUP-AGG

In [ ]:
dfcwci.groupby("state").sum()

In [ ]:
dfcwci.groupby("state")['amount'].mean()

In [ ]:
dfcwci.state.unique()

In [ ]:
out=make_query("SELECT state,SUM(amount) FROM contributors GROUP BY state;")
make_frame(out, legend=['state','sum'])

###DELETE

In [ ]:
dfcwci.head()

In-place drops

In [ ]:
df2=dfcwci.copy()
df2.set_index('last_name', inplace=True)
df2.head()

In [ ]:
df2.drop(['Ahrens'], inplace=True)
df2.head()

In [ ]:
df2.reset_index(inplace=True)
df2.head()

The recommended way to do it is to create a new dataframe. This might be impractical is things are very large.

In [ ]:
dfcwci=dfcwci[dfcwci.last_name!='Ahrens']
dfcwci.head(10)

In [ ]:
drow="DELETE FROM contributors WHERE last_name=\"Ahrens\";"
db.cursor().execute(drow)

In [ ]:
db.commit()
out=make_query("SELECT * FROM contributors;")
make_frame(out).head(10)

###LIMIT

In [ ]:
out=make_query("SELECT * FROM contributors LIMIT 3;")
make_frame(out).head(10)

In [ ]:
dfcwci[0:3]

##Indexes

In [ ]:
crind="CREATE INDEX amount_ix ON contributors(amount);"
db.cursor().execute(crind)
db.commit()

In [ ]:
%%bash
echo ".schema" | sqlite3 cancont.db

In [ ]:
crind="DROP INDEX amount_ix;"
db.cursor().execute(crind)
db.commit()

In [ ]:
%%bash
echo ".schema" | sqlite3 cancont.db

##Relationships: JOINs are Cartesian Products.

###Simple subselect

In [ ]:
dfcand.head()

In [ ]:
obamaid=dfcand.query("last_name=='Obama'")['id'].values[0]

In [ ]:
obamacontrib=dfcwci.query("candidate_id==%i" % obamaid)
obamacontrib.head()

In [ ]:
russiandollsel="""
SELECT * FROM contributors WHERE 
    candidate_id = (SELECT id from candidates WHERE last_name = 'Obama');
"""
out=make_query(russiandollsel)
make_frame(out).head()

###implicit join

In [ ]:
implicitjoinsel="""
SELECT 
    contributors.last_name, contributors.first_name, contributors.amount, candidates.last_name 
FROM 
    contributors, candidates 
WHERE contributors.candidate_id = candidates.id
AND candidates.last_name = 'Obama';
"""
out=make_query(implicitjoinsel)
make_frame(out, legend=["contributors.last_name", 
            "contributors.first_name", "contributors.amount", "candidates.last_name"]).head()

Let's expand to not just include Obama

In [ ]:
implicitjoinsel="""
SELECT 
    contributors.last_name, contributors.first_name, contributors.amount, candidates.last_name 
FROM 
    contributors, candidates 
WHERE contributors.candidate_id = candidates.id;
"""
out=make_query(implicitjoinsel)
make_frame(out, legend=["contributors.last_name", 
            "contributors.first_name", "contributors.amount", "candidates.last_name"]).head()

###Explicit INNER JOIN

![inner join](http://pandas.pydata.org/pandas-docs/stable/_images/merging_merge_on_key_inner.png)

(from http://pandas.pydata.org/pandas-docs/stable/merging.html)

In [ ]:
cols_wanted=['last_name_x', 'first_name_x', 'candidate_id', 'id', 'last_name_y']
dfcwci.merge(dfcand, left_on="candidate_id", right_on="id")[cols_wanted]

In [ ]:
explicitjoinsel="""
SELECT 
    contributors.last_name, contributors.first_name, candidates.last_name 
FROM 
    contributors JOIN candidates 
ON contributors.candidate_id = candidates.id;
"""
out=make_query(explicitjoinsel)
make_frame(out, legend=["contributors.last_name", 
            "contributors.first_name",  "candidates.last_name"]).head()

In [ ]:
explicitjoinsel="""
SELECT 
    COUNT(contributors.id), contributors.first_name, candidates.last_name 
FROM 
    contributors JOIN candidates 
ON contributors.candidate_id = candidates.id

GROUP BY candidates.last_name;
"""
out=make_query(explicitjoinsel)
make_frame(out, legend=["count(contributors.id)", 
            "contributors.first_name",  "candidates.last_name"])

In [ ]:
len(make_query("SELECT DISTINCT id, last_name FROM candidates;"))

###Outer JOIN

####left outer (contributors on candidates)

![left outer](http://pandas.pydata.org/pandas-docs/stable/_images/merging_merge_on_key_left.png)

In [ ]:
dfcwci.merge(dfcand, left_on="candidate_id", right_on="id", how="left")[cols_wanted]

In [ ]:
explicitjoinsel="""
SELECT 
    COUNT(contributors.id), contributors.first_name, candidates.last_name,
        contributors.candidate_id, candidates.id
FROM 
    contributors LEFT OUTER JOIN candidates 
ON contributors.candidate_id = candidates.id

GROUP BY candidates.last_name;
"""
out=make_query(explicitjoinsel)
make_frame(out, legend=["count(contributors.id)", "contributors.first_name",  
            "contributors.candidate_id", "candidates.id", "candidates.last_name"])

####right outer (contributors on candidates) = left outer (candidates on contributors)

We do it thus as sqlite has no support for right outer or plain outer. If it did we could write:

```sql
SELECT 
    COUNT(contributors.id), contributors.first_name, candidates.last_name 
FROM 
    contributors RIGHT OUTER JOIN candidates 
ON contributors.candidate_id = candidates.id

GROUP BY candidates.last_name;
```

![right outer](http://pandas.pydata.org/pandas-docs/stable/_images/merging_merge_on_key_right.png)

In [ ]:
dfcwci.merge(dfcand, left_on="candidate_id", right_on="id", how="right")[cols_wanted]

In [ ]:
explicitjoinsel="""
SELECT 
    COUNT(contributors.id), contributors.first_name, candidates.last_name, 
        contributors.candidate_id, candidates.id
FROM 
    candidates LEFT OUTER JOIN contributors 
ON contributors.candidate_id = candidates.id

GROUP BY candidates.last_name;
"""
out=make_query(explicitjoinsel)
make_frame(out, legend=["count(contributors.id)", "contributors.first_name",  
                    "contributors.candidate_id", "candidates.id", "candidates.last_name"])

####full outer

also not supported by sqlite

```sql
SELECT 
    COUNT(contributors.id), contributors.first_name, candidates.last_name 
FROM 
    contributors FULL OUTER JOIN candidates 
ON contributors.candidate_id = candidates.id

GROUP BY candidates.last_name;
```

![outer](http://pandas.pydata.org/pandas-docs/stable/_images/merging_merge_on_key_outer.png)

In [ ]:
dfcwci.merge(dfcand, left_on="candidate_id", right_on="id", how="outer")[cols_wanted]

When to use which?

See this:

http://blog.codinghorror.com/a-visual-explanation-of-sql-joins/

##Pandas /SQL

In [ ]:
pd.read_sql("SELECT * FROM candidates WHERE party= 'D';", db)

In [ ]:
pd.read_sql(implicitjoinsel, db)

This is very useful if the database is big and out of memory. Sqlite3 is the only db2api database supported. For any other database you should use `SQLAlchemy`. See, for eg: https://plot.ly/ipython-notebooks/big-data-analytics-with-pandas-and-sqlite/

In [ ]:
db.close()

##Useful Links

- http://sebastianraschka.com/Articles/sqlite3_database.html and  http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html#unique_indexes
- https://github.com/tthibo/SQL-Tutorial
- chrisalbon.com

And especially for R users:

- https://cran.r-project.org/web/packages/dplyr/vignettes/introduction.html
- https://gist.github.com/TomAugspurger/6e052140eaa5fdb6e8c0/